# 0: Load modules and functions

In [190]:
#necessary modules for the mathematica link function
import os
import subprocess

# other stuff
import pandas as pd
import doctest 

# https://mathematica.stackexchange.com/questions/4643/how-to-use-mathematica-functions-in-python-programs
# ^ used instructions in this link

In [59]:
def execute_terminal(cmd='echo "Hello World!"',v=False):
    """
    Takes a command in the form of 'run "<mathematica func>"'
    >>> int(execute_terminal('run Prime[100]'))
    541
    
    >>> execute_terminal('run "Sum[2x-1,{x,1,k}]"')
    'k^2'
    
    >>> execute_terminal('run Integrate[Log[x],x]')
    '-x + x*Log[x]'
    
    >>> execute_terminal("run 'Zeta[2]'")
    'Pi^2/6'
    
    """
    if v: print(cmd)
    proc=subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, )
    if v: print(proc)
    output=proc.communicate()[0].decode('utf-8')[:-1]
    if v: print(output)
    return output
execute_terminal()

'Hello World!'

In [165]:
def _check_double_signs(string):
    if "--" in string:
        string = string.replace("--",'+')
    return string

In [166]:
def Mrun(mathematica_command="Prime[100]",v=False):
    #add checks to commands!
    mathematica_command = _check_double_signs(mathematica_command)
    
    
    output = execute_terminal('run "{}"'.format(mathematica_command.replace(" ",'')),v)
    if v: print(output)
    return output

In [196]:
def find_func_extrema(func,max_extrema=True,v=False):
    """
    Find maximum or minimum of a function.
    >>> find_func_extrema('-x^3 + 3*x^2 - 4',max_extrema=False)
    Minimum point: (x=-1,y=-6)
    
    >>> find_func_extrema('-x^3 + 3*x^2 - 4',max_extrema=True)
    Maximum point: (x=2,y=0)
    """
    
    #TODO: Add functionality so that multiple y_val minimums are appended to a list
    
    extrema_to_be_found = 'Maximum' if max_extrema else "Minimum"
    
    ## get x values for extrema
    #find derivative of function
    d_func = Mrun(f"D[{func}]")
    # get x values of where derivative is 0
    extrema_func_vals = make_output_array_list(Mrun(f"Solve[{d_func}==0,x]"))
    
    
    ## find evaluated extrema
    extrema_yval = None
    extrema_xval = None
    
    
    
    for x_val in extrema_func_vals:
        #replace variables in function with number
        mod_func = func.replace('x',x_val)
        
        #evaluate the function with numbers in it
        y_val = Mrun(f"Evaluate[{mod_func}]")
        if v: print(f"when x_val --> {x_val}, y_val -->{y_val}")
        if extrema_yval == None and extrema_xval==None: extrema_yval, extrema_xval = y_val,x_val
        variable_change_condition = extrema_yval < y_val if max_extrema else extrema_yval > y_val
        
        
            
            
        if variable_change_condition:
            extrema_yval = y_val
            extrema_xval = x_val

    if v: print(f"{extrema_to_be_found} point: (x={extrema_xval},y={extrema_yval})")
    return extrema_xval,extrema_yval

In [116]:
def make_output_array_list(string,dimension=1):
    #TODO: first check dimension ... etc. useful for highdimensional data
    
    #remove stuff
    string = string.replace(' ','') # remove white space from string
    string = string.replace("{",'').replace("}",'') #remove the {} symbols
    string = string.replace("->",'') # remove arrows from output
    string = string.replace("x",'') #remove variable 'x' from output
    
    #make list
    string = string.split(',') #make a list splitting on commas
    
    return string
    
    

In [193]:
doctest.testmod()

TestResults(failed=0, attempted=6)

# 1: Practice Simple Calculus
* show that this works for `integration` and `differentiation`
* show that this works for 2D `optimizaiton`

### Differentiation

**Rating: 9/10**

In [247]:
#problem 1, find the derivative of a function
funcs = ['x^2', '3 - 14x', '10 + 5x-x^2', 
         'x^3-2x^2+x-1','Sqrt[3x-4]',
         'Sqrt[1-9x]'] 
results = []
for f in funcs:
    results.append(Mrun(f"D[{f},x]",v=False)) # this function fails if the inputted function has spaces in it
results

['2*x',
 '-14',
 '5 - 2*x',
 '1 - 4*x + 3*x^2',
 '3/(2*Sqrt[-4 + 3*x])',
 '-9/(2*Sqrt[1 - 9*x])']

### Integration
**Rating: 9/10**

In [96]:
# problem 1, integrate the previous results
integrals = []
for n in results:
    integrals.append(Mrun(f"Integrate[{n},x]",v=False))
integrals

['x^2',
 '-14*x',
 '5*x - x^2',
 'x - 2*x^2 + x^3',
 'Sqrt[-4 + 3*x]',
 'Sqrt[1 - 9*x]']

### Optimization
**Rating: 7/10**
* mostly works, but there might be some minor issues to work out, like in Problem 3

In [245]:
# PROBLEM 1
# h(x) = -x^3 + 3x^2 -4 
# find values of x where h has maximum. then, find minimum. 

func = '-x^3 + 3*x^2 - 4'
find_func_extrema(func,v=True, max_extrema=True) # maximum val
print()
find_func_extrema(func,v=True, max_extrema=False) #minimum val

#answer: max --> x=2
        #min --> x=-1

when x_val --> -1, y_val -->-6
when x_val --> 2, y_val -->0
when x_val --> 2, y_val -->0
Maximum point: (x=2,y=0)

when x_val --> -1, y_val -->-6
when x_val --> 2, y_val -->0
when x_val --> 2, y_val -->0
Minimum point: (x=-1,y=-6)


('-1', '-6')

In [200]:
# PROBLEM 2
# if g(x) is a function, and its derivative is x^5 * (x+1)(x-1), 
# how many minima does g(x) have?

dg_func = 'x^5 * (x+1)*(x-1)'
g_func = Mrun(f"Integrate[{dg_func},x]")

find_func_extrema(g_func, v=True, max_extrema=False) 
#answer: 2 minimum

when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> -2/Sqrt[3], y_val -->1/108
when x_val --> 2/Sqrt[3], y_val -->-1/108
Minimum point: (x=2/Sqrt[3],y=-1/108)


('2/Sqrt[3]', '-1/108')

In [242]:
# PROBLEM 3
# for what value of x does g have a minimum if g = x^6 - 3x^5
g_func = 'x^6 - 3*(x)^(5)'
#find_func_extrema(g_func, max_extrema=False,v=True) # my function isn't working...

#....but i just realized mathematica actually has a function pre made for this. now I feel dumb.

Mrun("FindMinimum[(%s),{{x,1.5}}]"%g_func)


In [246]:
# PROBLEM 4
# at how many points does f have a local max, if deriv(F) is x^4(x-2)(x+3)?
df_func = "x^4*(x-2)*(x+3)"
f_func = Mrun(f"Integrate[{df_func},x]")
find_func_extrema(f_func,max_extrema=True, v=True)

when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> 0, y_val -->0
when x_val --> (-35-Sqrt[31465])/60, y_val -->(71*(-35 - Sqrt[31465]))/19595520000000 + (35 + Sqrt[31465])/648000000
when x_val --> (-35+Sqrt[31465])/60, y_val -->(35 - Sqrt[31465])/648000000 + (71*(-35 + Sqrt[31465]))/19595520000000
Maximum point: (x=0,y=0)


('0', '0')